# **HW1: Regression**
In *assignment 1*, you need to finish:

1.  Basic Part: Implement two regression models to predict the Systolic blood pressure (SBP) of a patient. You will need to implement **both Matrix Inversion and Gradient Descent**.


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implement one regression model to predict the SBP of multiple patients in a different way than the basic part. You can choose **either** of the two methods for this part.

# **1. Basic Part (55%)**
In the first part, you need to implement the regression to predict SBP from the given DBP


## 1.1 Matrix Inversion Method (25%)


*   Save the prediction result in a csv file **hw1_basic_mi.csv**
*   Print your coefficient


### *Import Packages*

> Note: You **cannot** import any other package

In [221]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

### *Global attributes*
Define the global attributes

In [222]:
training_dataroot = 'hw1_basic_training.csv' # Training data file file named as 'hw1_basic_training.csv'
testing_dataroot = 'hw1_basic_testing.csv'   # Testing data file named as 'hw1_basic_training.csv'
output_dataroot = 'hw1_basic_mi.csv' # Output file will be named as 'hw1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be 20 * 3 matrix and saved as numpy array
                      # The format of each row should be ['subject_id', 'charttime', 'sbp']

You can add your own global attributes here


In [223]:
training_dataset_1 = []     # The first 80% of training_datalist
validation_dataset_1 = []   # The last 20% of training_datalist

training_dataset_2 = []     # Randomly selected 80% of training_datalist
validation_dataset_2 = []   # The rest of training_datalist

training_dataset_3 = []     # Randomly selected 70% of training_datalist
validation_dataset_3 = []   # The rest of training_datalist

training_dataset_4 = []     # Randomly selected 60% of training_datalist
validation_dataset_4 = []   # The rest of training_datalist

training_data_num = 0

def MAPE(y_ans, y_prediction):
    return np.mean(np.abs((y_ans - y_prediction) / y_ans)) * 100

### *Load the Input File*
First, load the basic input file **hw1_basic_training.csv** and **hw1_basic_testing.csv**

Input data would be stored in *training_datalist* and *testing_datalist*

In [224]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
	global training_datalist, training_data_num
	
	training_datalist = np.array(list(csv.reader(csvfile)))
	training_datalist = np.delete(training_datalist, 0, 0)
	training_data_num = training_datalist.shape[0]

with open(testing_dataroot, newline='') as csvfile:
	global testing_datalist

	testing_datalist = np.array(list(csv.reader(csvfile)))
	testing_datalist = np.delete(testing_datalist, 0, 0)

### *Implement the Regression Model*

> Note: It is recommended to use the functions we defined, you can also define your own functions


#### Step 1: Split Data
Split data in *training_datalist* into training dataset and validation dataset
* Validation dataset is used to validate your own model without the testing data



In [225]:
def SplitData():
    def RandomSplitData(propotion):
        # We have to deepcopy the training_datalist since we don't want to change the original one
        training_datalist_copy = training_datalist.copy()
        
        random_times = random.randint(1, 10)
        for i in range(random_times):
            # Since the copy is a numpy array, we have to use np.random.shuffle instead of random.shuffle
            # Or it will have some repeated elements after shuffling
            np.random.shuffle(training_datalist_copy)

        return training_datalist_copy[ : propotion], training_datalist_copy[propotion : ]

    # We have to minus 1 since the first row is the name of each column
    global training_data_num
    propotion_80 = int(training_data_num * 0.8 + 0.5)   # 0.5 is for rounding
    propotion_70 = int(training_data_num * 0.7 + 0.5)
    propotion_60 = int(training_data_num * 0.6 + 0.5)
    
    global training_datalist
    global training_dataset_1, validation_dataset_1
    global training_dataset_2, validation_dataset_2
    global training_dataset_3, validation_dataset_3 
    global training_dataset_4, validation_dataset_4

    training_dataset_1 = training_datalist[0 : propotion_80 ]
    validation_dataset_1 = training_datalist[propotion_80 : ]
    training_dataset_2, validation_dataset_2 = RandomSplitData(propotion_80)
    training_dataset_3, validation_dataset_3 = RandomSplitData(propotion_70)
    training_dataset_4, validation_dataset_4 = RandomSplitData(propotion_60)

#### Step 2: Preprocess Data
Handle the unreasonable data
> Hint: Outlier and missing data can be handled by removing the data or adding the values with the help of statistics  

In [226]:
def PreprocessData():
    '''
    # ref
    1. https://andy6804tw.github.io/2021/04/02/python-outliers-clean/ \n
    2. https://chat.openai.com/share/76e7e9a6-7e42-4cdc-a2bb-72af60082814

    By searching some technique, I decided to remove the outliers by using IQR
    Since I roughly check the training data and found that the outliers are not too much
    If just one of the number in the row is outlier, I will modify its value based on another number in the same row
    '''
    
    global training_datalist, training_data_num
    dbp_list = np.array([float(training_datalist[i][0]) for i in range(training_data_num)])
    sbp_list = np.array([float(training_datalist[i][1]) for i in range(training_data_num)])

    dbp_IQR = np.percentile(dbp_list, 75) - np.percentile(dbp_list, 25)
    sbp_IQR = np.percentile(sbp_list, 75) - np.percentile(sbp_list, 25)

    dbp_upper = np.percentile(dbp_list, 75) + 1.5 * dbp_IQR
    dbp_lower = np.percentile(dbp_list, 25) - 1.5 * dbp_IQR
    sbp_upper = np.percentile(sbp_list, 75) + 1.5 * sbp_IQR
    sbp_lower = np.percentile(sbp_list, 25) - 1.5 * sbp_IQR

    tem_list = list()
    for i in range(training_data_num):
        diff_with_mean_dbp = (dbp_list[i] - np.mean(dbp_list)) / dbp_IQR
        diff_with_mean_sbp = (sbp_list[i] - np.mean(sbp_list)) / sbp_IQR
        is_outlier_dbp = bool(dbp_list[i] > dbp_upper or dbp_list[i] < dbp_lower)
        is_outlier_sbp = bool(sbp_list[i] > sbp_upper or sbp_list[i] < sbp_lower)

        if is_outlier_dbp and is_outlier_sbp:
            continue
        elif is_outlier_dbp:
            dbp_list[i] = int(np.mean(dbp_list) + diff_with_mean_sbp * dbp_IQR + 0.5)
        elif is_outlier_sbp:
            sbp_list[i] = int(np.mean(sbp_list) + diff_with_mean_dbp * sbp_IQR + 0.5)

        # if is_outlier_dbp or is_outlier_sbp: 
        #     continue
        tem_list.append([dbp_list[i], sbp_list[i]])

    training_datalist = np.array(tem_list)
    training_data_num = training_datalist.shape[0]

#### Step 3: Implement Regression
> use Matrix Inversion to finish this part




In [227]:
def MatrixInversion():
    global training_dataset_1
    global training_dataset_2
    global training_dataset_3
    global training_dataset_4

    training_dataset_list = []
    training_dataset_list.append(training_dataset_1)
    training_dataset_list.append(training_dataset_2)
    training_dataset_list.append(training_dataset_3)
    training_dataset_list.append(training_dataset_4)

    # each element contains [phi, phi_T, y]
    container = [[
        np.array([[1, float(training_dataset_list[i][j][0])] for j in range(len(training_dataset_list[i]))]),
        np.array([[1, float(training_dataset_list[i][j][0])] for j in range(len(training_dataset_list[i]))]).T,
        np.array([float(training_dataset_list[i][j][1]) for j in range(len(training_dataset_list[i]))])
    ] for i in range(len(training_dataset_list))]

    w = [ np.dot( np.dot( np.linalg.inv(np.dot(container[i][1], container[i][0])), container[i][1]), container[i][2]).tolist() for i in range(len(container)) ]
    return [sum(row[0] for row in w) / len(w), sum(row[1] for row in w) / len(w)]

#### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*
The final *output_datalist* should look something like this 
> [ [100], [80], ... , [90] ] where each row contains the predicted SBP

In [228]:
def MakePrediction(coefficients):
    global testing_datalist, output_datalist

    output_datalist = np.dot(
        np.array([ [1, int(testing_datalist[i][0])] for i in range(len(testing_datalist)) ]),
        np.array([[coefficients[0]], [coefficients[1]]])
    )

    # testing_dbp = np.array([ [1, int(testing_datalist[i][0])] for i in range(len(testing_datalist)) ])
    # coe = np.array([[coefficients[0]], [coefficients[1]]])
    # result = np.dot(testing_dbp, coe)


#### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted
* If your regression model is *3x^2 + 2x^1 + 1*, your output would be:
```
3 2 1
```





In [229]:
def Validation(datalist, coefficients):
    prediction = np.dot(
        np.array([ [1, int(datalist[i][0])] for i in range(len(datalist)) ]),
        np.array([[coefficients[0]], [coefficients[1]]])
    )

    return MAPE(np.array([float(datalist[i][1]) for i in range(len(datalist))]), prediction)
    

PreprocessData()
SplitData()
print(f'after preprocessing : {training_datalist.shape[0]}')
print(f'first training list num : {training_dataset_1.shape[0]}')
print(f'first validation list num : {validation_dataset_1.shape[0]}\n')
print(f'second training list num : {training_dataset_2.shape[0]}')
print(f'second validation list num : {validation_dataset_2.shape[0]}\n')
print(f'third training list num : {training_dataset_3.shape[0]}')
print(f'third validation list num : {validation_dataset_3.shape[0]}\n')
print(f'fourth training list num : {training_dataset_4.shape[0]}')
print(f'fourth validation list num : {validation_dataset_4.shape[0]}')
print('\n' + '-' * 30)

coefficients = MatrixInversion()
print(coefficients[1], coefficients[0])
print('\n' + '-' * 30)
print(f'MAPE for training set 1 : {Validation(validation_dataset_1, coefficients)}')
print(f'MAPE for training set 2 : {Validation(validation_dataset_2, coefficients)}')
print(f'MAPE for training set 3 : {Validation(validation_dataset_3, coefficients)}')
print(f'MAPE for training set 4 : {Validation(validation_dataset_4, coefficients)}')

# MakePrediction(coefficients)
# print(output_datalist)

after preprocessing : 373
first training list num : 298
first validation list num : 75

second training list num : 298
second validation list num : 75

third training list num : 261
third validation list num : 112

fourth training list num : 224
fourth validation list num : 149

------------------------------
0.9769529294792862 48.618843763005756

------------------------------
MAPE for training set 1 : 12.979509241449186
MAPE for training set 2 : 12.18126125319156
MAPE for training set 3 : 13.023998568049239
MAPE for training set 4 : 11.97945093598516


### *Write the Output File*
Write the prediction to output csv
> Format: 'sbp'




In [230]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
	writer = csv.writer(csvfile)
	for row in output_datalist:
		writer.writerow(row)

## 1.2 Gradient Descent Method (30%)


*   Save the prediction result in a csv file **hw1_basic_gd.csv**
*   Output your coefficient update in a csv file **hw1_basic_coefficient.csv**
*   Print your coefficient





### *Global attributes*

In [231]:
output_dataroot = 'hw1_basic_gd.csv' # Output file will be named as 'hw1_basic.csv'
coefficient_output_dataroot = 'hw1_basic_coefficient.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be 20 * 3 matrix and saved as numpy array
                      # The format of each row should be ['subject_id', 'charttime', 'sbp']

coefficient_output = [] # Your coefficient update during gradient descent
                   # Should be a (number of iterations * number_of coefficient) matrix
                   # The format of each row should be ['w0', 'w1', ...., 'wn']

Your own global attributes

In [232]:
# We reopen the file since we have modified the data in the previous part
with open(training_dataroot, newline='') as csvfile:
	global training_datalist, training_data_num
	
	training_datalist = np.array(list(csv.reader(csvfile)))
	training_datalist = np.delete(training_datalist, 0, 0)
	training_data_num = training_datalist.shape[0]

with open(testing_dataroot, newline='') as csvfile:
	global testing_datalist

	testing_datalist = np.array(list(csv.reader(csvfile)))
	testing_datalist = np.delete(testing_datalist, 0, 0)

training_list_1 = []
validation_list_1 = []

training_list_2 = []
validation_list_2 = []

### *Implement the Regression Model*


#### Step 1: Split Data

In [233]:
def SplitData():
    global training_datalist
    global training_list_1, validation_list_1
    global training_list_2, validation_list_2
    
    _80_percent = int(training_data_num * 0.8 + 0.5)
    _90_percent = int(training_data_num * 0.9 + 0.5)

    training_list_1 = training_datalist[ : _80_percent]
    validation_list_1 = training_datalist[_80_percent : ]

    training_list_2 = training_datalist[ : _90_percent]
    validation_list_2 = training_datalist[_90_percent : ]

#### Step 2: Preprocess Data

In [234]:
def PreprocessData():
    global training_datalist

    training_nums = len(training_datalist)
    dbp_list = np.array([float(training_datalist[i][0]) for i in range(training_nums)])
    sbp_list = np.array([float(training_datalist[i][1]) for i in range(training_nums)])

    dbp_std = np.std(dbp_list)
    dbp_mean = np.mean(dbp_list)
    sbp_std = np.std(sbp_list)
    sbp_mean = np.mean(sbp_list)

    tem = [
        [dbp_list[i], sbp_list[i]] 
        for i in range(training_nums) if (dbp_list[i] < dbp_mean + 2 * dbp_std and dbp_list[i] > dbp_mean - 2 * dbp_std) and (sbp_list[i] < sbp_mean + 2 * sbp_std and sbp_list[i] > sbp_mean - 2 * sbp_std)
    ]
    print(len(tem))
    training_datalist = np.array(tem)

#### Step 3: Implement Regression
> use Gradient Descent to finish this part

In [235]:
def GradientDescent(learning_rate):
    def gradient_of_loss(y_ans, y_prediction, x):
        return (-2 * (y_ans - y_prediction)).dot(x)
    
    global training_list_1
    global training_list_2
    training_set = []
    training_set.append(training_list_1)
    # training_set.append(training_list_2)

    global validation_list_1
    global validation_list_2
    validation_set = []
    validation_set.append(validation_list_1)
    validation_set.append(validation_list_2)

    nums = len(training_list_1)
    coefficient = np.array([[30], [1.2]])
    x = np.array( [[1, training_list_1[i][0]] for i in range(nums)] )
    y_ans = np.array( [[training_list_1[i][1]] for i in range(nums)] )
    y_prediction = x.dot(coefficient)

    print(y_ans.shape, y_prediction.shape, x.shape)
    gradient = gradient_of_loss(y_ans.T, y_prediction.T, x)
    # print((learning_rate * gradient).shape, gradient.shape, coefficient.shape)
    # print(coefficient - learning_rate * gradient.T)
    print(gradient[0][1] > 0.01)
    while (gradient[0][1] > 0.0002) or (gradient[0][1] < -0.0002):
        # print(' '.join(map(str, gradient)))
        coefficient -= learning_rate * gradient.T
        y_prediction = x.dot(coefficient)
        gradient = gradient_of_loss(y_ans.T, y_prediction.T, x)
    print(' '.join(map(str, gradient)))

    coefficient -= learning_rate * gradient.T
    y_prediction = (x.dot(coefficient)).astype(int)
    # print(' '.join(map(str, y_prediction)))
    print('\n' + '-' * 30)
    print(f'MAPE for training list 1 : {MAPE(y_ans, y_prediction)}')
    print(f'MAPE for validation list 1 : {MAPE(np.array( [[validation_list_1[i][1]] for i in range(len(validation_list_1))]), (np.array( [[1, validation_list_1[i][0]] for i in range(len(validation_list_1))] ).dot(coefficient)) )}')
    print(f'MAPE for validation list 2 : {MAPE(np.array( [[validation_list_2[i][1]] for i in range(len(validation_list_2))]), (np.array( [[1, validation_list_2[i][0]] for i in range(len(validation_list_2))] ).dot(coefficient)) )}')

#### Step 4: Make Prediction

Make prediction of testing dataset and store the values in *output_datalist*
The final *output_datalist* should look something like this 
> [ [100], [80], ... , [90] ] where each row contains the predicted SBP

Remember to also store your coefficient update in *coefficient_output*
The final *coefficient_output* should look something like this
> [ [1, 0, 3, 5], ... , [0.1, 0.3, 0.2, 0.5] ] where each row contains the [w0, w1, ..., wn] of your coefficient





In [236]:
def MakePrediction():
    pass

#### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted
* If your regression model is *3x^2 + 2x^1 + 1*, your output would be:
```
3 2 1
```



In [237]:
PreprocessData()
SplitData()
GradientDescent(0.000000475)

370
(298, 1) (298, 1) (298, 2)
True


[-0.01681781  0.0002    ]

------------------------------
MAPE for training list 1 : 5.296185720883669
MAPE for validation list 1 : 5.696416643952454
MAPE for validation list 2 : 6.4556076335961405


### *Write the Output File*

Write the prediction to output csv
> Format: 'sbp'

**Write the coefficient update to csv**
> Format: 'w0', 'w1', ..., 'wn'
>*   The number of columns is based on your number of coefficient
>*   The number of row is based on your number of iterations

In [238]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
	writer = csv.writer(csvfile)
	for row in output_datalist:
		writer.writerow(row)

with open(coefficient_output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
	writer = csv.writer(csvfile)
	for row in coefficient_output:
		writer.writerow(row)

# **2. Advanced Part (40%)**
In the second part, you need to implement the regression in a different way than the basic part to help your predictions of multiple patients SBP.

You can choose **either** Matrix Inversion or Gradient Descent method.

The training data will be in **hw1_advanced_training.csv** and the testing data will be in **hw1_advanced_testing.csv**.

Output your prediction in **hw1_advanced.csv**

Notice:
> You cannot import any other package other than those given



### Input the training and testing dataset

In [239]:
training_dataroot = 'hw1_advanced_training.csv' # Training data file file named as 'hw1_basic_training.csv'
testing_dataroot = 'hw1_advanced_testing.csv'   # Testing data file named as 'hw1_basic_training.csv'
output_dataroot = 'hw1_advanced.csv' # Output file will be named as 'hw1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be 220 * 1 matrix and saved as numpy array
                      # The format of each row should be ['sbp']

### Your Implementation

In [240]:
def read_data():
    train_id = []
    train_time = []
    train_data = []

    training_data = []; train_nums = 0
    with open(training_dataroot, newline='') as csvfile:
        training_data = list(csv.reader(csvfile))[1 : ]

    train_id = [int(data[0]) for data in training_data]

    train_data = np.array([data[2 : ] for data in training_data])
    train_data[train_data == ''] = np.nan
    train_data = train_data.astype(float)

    for data in training_data:
        time_list = data[1].split(' ')
        days_ago = int(time_list[0])

        time = time_list[2].split(':')
        hour, minute = int(time[0]), int(time[1])
        train_time.append([days_ago, hour, minute])

    return train_id, train_time, train_data

In [241]:
def preprocess(train_id, train_time, train_data):
    # remove the data with more than 2 elements missing
    i = 0; train_nums = len(train_data)
    while i < train_nums:
        if np.sum(np.isnan(train_data[i])) >= 2:
            train_data = np.delete(train_data, i, 0)
            train_id.pop(i)
            train_time.pop(i)
            train_nums -= 1
            i -= 1
        i += 1
    print(train_data.shape, len(train_id), len(train_time))

    for data in train_data:
        if data[1] == np.nan or data[2] == np.nan or data[3] == np.nan:
            print(data)
            break
        elif data[0] == np.nan:
            print(data)

    # remove outliers
    ## We first divide the data into group_1 by id
    # last_id = train_id[0]; group_1 = []; tem_group = []
    # for i, data in enumerate(train_id):
    #     tem_group.append(train_data[i])
    #     if data != last_id:
    #         group_1.append(tem_group)
    #         tem_group = []
    #         last_id = data
    # group_1.append(tem_group)

    ## Then we 


preprocess(*read_data())


(5566, 5) 5566 5566


### Output your Prediction

> your filename should be **hw1_advanced.csv**

In [242]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
	writer = csv.writer(csvfile)
	for row in output_datalist:
		writer.writerow(row)

# Report *(5%)*

Report should be submitted as a pdf file **hw1_report.pdf**

*   Briefly describe the difficulty you encountered
*   Summarize your work and your reflections
*   No more than one page






# Save the Code File
Please save your code and submit it as an ipynb file! (**hw1.ipynb**)